<a href="https://colab.research.google.com/github/heghiw/lakovaci-linka/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Remove the existing directory
!rm -rf lakovaci-linka

# Clone the repository again
!git clone https://github.com/heghiw/lakovaci-linka.git

# Navigate to the directory
%cd lakovaci-linka


Cloning into 'lakovaci-linka'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 52 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 9.26 MiB | 14.45 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/lakovaci-linka


data prep

In [2]:
import pandas as pd

# Define the path to the Excel file
file_path = '/content/lakovaci-linka/data1.xlsx'

# Read the sheet 'linka' into a DataFrame
linka_df = pd.read_excel(file_path, sheet_name='linka')

# Read the sheet 'recept' into a DataFrame
recept_df = pd.read_excel(file_path, sheet_name='recept')

print("Linka:")
print(linka_df)
print("\nRecepty:")
print(recept_df)



Linka:
             typ vany  id_vany   pozice_x_rel  pozice_x_cum
0               vstup         0           0.0           0.0
1        Teplý oplach         1        2300.0        2300.0
2             Postřik         2        1900.0        4200.0
3    Ponor odm železo         3        1900.0        6100.0
4    Ponor odm pozink         4        1800.0        7900.0
5            oplach 1         5        1800.0        9700.0
6            oplach 2         6        1800.0       11500.0
7       Moření železo         7        1800.0       13300.0
8       Moření pozink         8        1800.0       15100.0
9      Oplach moř žel         9        1800.0       16900.0
10  Oplach moř pozink        10        1800.0       18700.0
11    oplach společný        11        1800.0       20500.0
12           aktivace        12        1800.0       22300.0
13             fosfát        13        1800.0       24100.0
14             oplach        14        1800.0       25900.0
15            Oplach         21  

manipulator and vozik characteristics

In [24]:
manipulator_characteristics = {
        "okap_point": 2000,
        "highest_point": 2750,
        "deceleration_point": 500,
        "speed_after_ponoreni": 8,
        "speed_going_up": 15,
        "speed_going_down": 12,
        "speed_left_right": 35
    }

In [3]:
linka_df.columns = linka_df.columns.str.strip()
recept_df.columns = recept_df.columns.str.strip()
print(linka_df.columns)
print(recept_df.columns)

Index(['typ vany', 'id_vany', 'pozice_x_rel', 'pozice_x_cum'], dtype='object')
Index(['tech', 'id_vany', 'poradi_operace', 'cas_min', 'cas_max', 'cas_opt',
       'okap', 'okap_cas'],
      dtype='object')


simulace


In [29]:
import pandas as pd
import numpy as np

ENTRY_POSITION = 0
EXIT_POSITION = 100

def assign_manipulator_ranges(linka_df, n_manipulators):
    positions = linka_df[['id_vany', 'pozice_x_cum']].dropna().sort_values('pozice_x_cum')
    clusters = np.array_split(positions, n_manipulators)
    return {i: (group['id_vany'].min(), group['id_vany'].max()) for i, group in enumerate(clusters)}

def find_manip_for_bath(bath_id, linka_df, ranges):
    x = linka_df.loc[linka_df['id_vany'] == bath_id, 'pozice_x_cum'].values[0]
    for m, (start, end) in ranges.items():
        ids = linka_df[linka_df['id_vany'].between(start, end)]['pozice_x_cum']
        if ids.min() <= x <= ids.max():
            return m
    raise ValueError("No manipulator found for bath.")

def compute_move_time(frm, to, linka_df, speeds):
    x1 = linka_df.loc[linka_df['id_vany'] == frm, 'pozice_x_cum'].values[0]
    x2 = linka_df.loc[linka_df['id_vany'] == to, 'pozice_x_cum'].values[0]
    dist = abs(x1 - x2) / 1000  # m
    horiz_speed = speeds['speed_left_right'] / 60  # m/s
    vert_down = speeds['speed_going_down'] / 60
    vert_up = speeds['speed_going_up'] / 60
    total_vert = speeds['highest_point'] / 1000
    return dist / horiz_speed + total_vert * (1 / vert_down + 1 / vert_up)

def build_true_pipelined_schedule(recept_df, tech, linka_df, speeds, n_manipulators, takt=300):
    recipe = recept_df[recept_df['tech'] == tech].sort_values('poradi_operace')
    manip_ranges = assign_manipulator_ranges(linka_df, n_manipulators)

    tasks = []
    manip_free = {m: 0.0 for m in range(n_manipulators)}
    bath_occupied = {}
    product_queue = []
    active_products = {}
    current_time = 0.0
    product_id = 0
    max_cycles = 2

    # Insert first product
    active_products[product_id] = {'step': 0, 'ready': 0.0}
    product_id += 1

    while True:
        scheduled = False

        for pid, state in active_products.items():
            step_idx = state['step']
            if step_idx >= len(recipe): continue
            step = recipe.iloc[step_idx]
            bath = step['id_vany']
            prev = ENTRY_POSITION if step_idx == 0 else recipe.iloc[step_idx - 1]['id_vany']
            manip = find_manip_for_bath(bath, linka_df, manip_ranges)
            move_time = compute_move_time(prev, bath, linka_df, speeds)
            dwell = step['cas_opt']
            okap = step['okap_cas'] if step['okap'] else 0

            ready = max(manip_free[manip], state['ready'])
            move_end = ready + move_time
            dwell_end = move_end + dwell
            okap_end = dwell_end + okap

            if bath in bath_occupied and bath_occupied[bath] > ready:
                continue

            tasks.append({
                'product': pid,
                'manipulator': manip,
                'step': step_idx,
                'from': prev,
                'to': bath,
                'start_time': ready,
                'end_time': move_end,
                'operation': 'move',
                'dwell_time': dwell,
                'cycle': pid // max_cycles
            })
            if okap:
                tasks.append({
                    'product': pid,
                    'manipulator': manip,
                    'step': step_idx,
                    'from': bath,
                    'to': bath,
                    'start_time': dwell_end,
                    'end_time': okap_end,
                    'operation': 'okap',
                    'dwell_time': 0,
                    'cycle': pid // max_cycles
                })

            manip_free[manip] = okap_end
            bath_occupied[bath] = dwell_end
            active_products[pid]['ready'] = okap_end
            active_products[pid]['step'] += 1
            scheduled = True

        if not scheduled:
            if product_id >= max_cycles:
                break
            active_products[product_id] = {'step': 0, 'ready': product_id * (takt / max_cycles)}
            product_id += 1

    df = pd.DataFrame(tasks)
    df['cycle_time'] = df.groupby('manipulator')['end_time'].transform('max')
    return df


In [30]:
def simulate_line_run(schedule_df, n_manipulators, n_products):
    rows = []
    for p in range(n_products):
        offset = p * (schedule_df['cycle_time'].max() / n_products)
        for _, row in schedule_df.iterrows():
            rows.append({
                'product': p,
                'manipulator': row['manipulator'],
                'operation': row['operation'],
                'from': row['from'],
                'to': row['to'],
                'start': row['start_time'] + offset,
                'end': row['end_time'] + offset,
                'cycle': p
            })
            if row['step'] == 0 and row['operation'] == 'move':
                rows.append({
                    'product': p,
                    'manipulator': None,
                    'operation': 'product_in',
                    'from': None,
                    'to': None,
                    'start': offset,
                    'end': offset,
                    'cycle': p
                })
            if row['to'] == EXIT_POSITION and row['operation'] == 'move':
                rows.append({
                    'product': p,
                    'manipulator': None,
                    'operation': 'product_out',
                    'from': None,
                    'to': None,
                    'start': row['end_time'] + offset,
                    'end': row['end_time'] + offset,
                    'cycle': p
                })
    return pd.DataFrame(rows)


In [31]:
def print_all_outputs(schedule_df, sim_df):
    print(f"\n=== Max Cycle Time: {schedule_df['cycle_time'].max():.2f} seconds ===")

    print("\n=== Full Cyclic Schedule ===")
    display(schedule_df)

    print("\n=== Schedule Per Manipulator ===")
    for m in schedule_df['manipulator'].unique():
        print(f"\n=== Manipulator {int(m)} Schedule ===")
        display(schedule_df[schedule_df['manipulator'] == m])

    print("\n=== Simulation Log ===")
    display(sim_df)


In [32]:
schedule_df = build_true_pipelined_schedule(
    recept_df, "tech1", linka_df, manipulator_characteristics, N_MANIPULATORS, takt=300
)
sim_df = simulate_line_run(schedule_df, N_MANIPULATORS, 10)
print_all_outputs(schedule_df, sim_df)


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)



=== Max Cycle Time: 4892.73 seconds ===

=== Full Cyclic Schedule ===


,product,manipulator,step,from,to,start_time,end_time,operation,dwell_time,cycle,cycle_time
0,0,0,0,0,0,0.000000,24.750000,move,0,0,1298.914286
1,0,0,1,0,1,24.750000,53.442857,move,120,0,1298.914286
2,0,0,1,1,1,173.442857,193.442857,okap,0,0,1298.914286
3,0,0,2,1,2,193.442857,221.450000,move,160,0,1298.914286
4,0,0,3,2,3,381.450000,409.457143,move,220,0,1298.914286
...,...,...,...,...,...,...,...,...,...,...,...
69,1,3,17,18,19,4626.364286,4654.200000,move,90,0,4864.035714
70,1,3,17,19,19,4744.200000,4745.200000,okap,0,0,4864.035714
71,1,3,18,19,20,4745.200000,4773.035714,move,90,0,4864.035714
72,1,3,18,20,20,4863.035714,4864.035714,okap,0,0,4864.035714



=== Schedule Per Manipulator ===

=== Manipulator 0 Schedule ===


,product,manipulator,step,from,to,start_time,end_time,operation,dwell_time,cycle,cycle_time
0,0,0,0,0,0,0.000000,24.750000,move,0,0,1298.914286
1,0,0,1,0,1,24.750000,53.442857,move,120,0,1298.914286
2,0,0,1,1,1,173.442857,193.442857,okap,0,0,1298.914286
3,0,0,2,1,2,193.442857,221.450000,move,160,0,1298.914286
4,0,0,3,2,3,381.450000,409.457143,move,220,0,1298.914286
5,0,0,3,3,3,629.457143,649.457143,okap,0,0,1298.914286
37,1,0,0,0,0,649.457143,674.207143,move,0,0,1298.914286
38,1,0,1,0,1,674.207143,702.900000,move,120,0,1298.914286
39,1,0,1,1,1,822.900000,842.900000,okap,0,0,1298.914286
40,1,0,2,1,2,842.900000,870.907143,move,160,0,1298.914286


...

,product,manipulator,step,from,to,start_time,end_time,operation,dwell_time,cycle,cycle_time
16,0,3,9,11,12,1597.892857,1625.728571,move,90,0,4864.035714
17,0,3,9,12,12,1715.728571,1735.728571,okap,0,0,4864.035714
18,0,3,10,12,13,1735.728571,1763.564286,move,330,0,4864.035714
19,0,3,10,13,13,2093.564286,2113.564286,okap,0,0,4864.035714
20,0,3,11,13,14,2113.564286,2141.400000,move,75,0,4864.035714
21,0,3,11,14,14,2216.400000,2231.400000,okap,0,0,4864.035714
22,0,3,12,14,21,2231.400000,2259.235714,move,75,0,4864.035714
23,0,3,12,21,21,2334.235714,2349.235714,okap,0,0,4864.035714
24,0,3,13,21,15,2349.235714,2377.071429,move,90,0,4864.035714
25,0,3,13,15,15,2467.071429,2487.071429,okap,0,0,4864.035714



=== Manipulator 5 Schedule ===


,product,manipulator,step,from,to,start_time,end_time,operation,dwell_time,cycle,cycle_time
36,0,5,19,20,100,3230.964286,3259.657143,move,0,0,4892.728571
73,1,5,19,20,100,4864.035714,4892.728571,move,0,0,4892.728571



=== Simulation Log ===


,product,manipulator,operation,from,to,start,end,cycle
0,0,0.0,move,0.0,0.0,0.000000,24.750000,0
1,0,NaN,product_in,NaN,NaN,0.000000,0.000000,0
2,0,0.0,move,0.0,1.0,24.750000,53.442857,0
3,0,0.0,okap,1.0,1.0,173.442857,193.442857,0
4,0,0.0,move,1.0,2.0,193.442857,221.450000,0
...,...,...,...,...,...,...,...,...
775,9,3.0,okap,19.0,19.0,9147.655714,9148.655714,9
776,9,3.0,move,19.0,20.0,9148.655714,9176.491429,9
777,9,3.0,okap,20.0,20.0,9266.491429,9267.491429,9
778,9,5.0,move,20.0,100.0,9267.491429,9296.184286,9
